Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [ ]:
## I think that this is generally a good solution, I only have some doubts that I'm gonna express. 

## Overall the code is well structured and written, the functions have clear names and purposes, and the comments help to understand the logic behind the code.
## The only comment that I have about the code is the fact that i would like some more information about the state of the solutions that
## you find running the simulated_annealing() function. It could be probably useful to print everytime you find a new best solution to
## have an idea of how the algorithm is working and how the solutions are evolving and improve the algorithm if we see that the algorithm
## is not finding many new best solutions after some time. I ran the code for some time and the fact that I see always a pretty similar
## output in terms of values of the best solution could indicate both that the algorithm is working well and converging to a good solution
## or that the algorithm is stuck in a local minima and is not able to explore the solution space enough. It could be easier to understand 
## with more stamps.

## It's a great solutions for the first two problems, the functions and the solution converges and the code is fast,
## but I have some doubts about the third problem. You always start with a random solutions, that is good for this problem, but
## I think that having only 10.000 iterations could not be enough to explore the solution space. In the third problem we have 5000
## items, so only adding or removing 1 item at a time could be not enough to explore the solution space well. My suggestion is to 
## increase the number of iterations when the number of items is high or to add/remove more than 1 item at a time. 

## Overall it seems to work pretty well.

In [1]:
import numpy as np

In [2]:
NUM_KNAPSACKS = 3
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [3]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [4]:
print(f"VALUES = {VALUES}")
print(f"WEIGHTS = {WEIGHTS}")
print(f"CONSTRAINTS = {CONSTRAINTS}")

VALUES = [ 8 55  8 84 60 61 31 79 71 57]
WEIGHTS = [[73 23]
 [61 93]
 [97 48]
 [30 51]
 [82 38]
 [30  1]
 [19 43]
 [ 3 25]
 [82 44]
 [62 58]]
CONSTRAINTS = [[169 240]
 [ 64 102]
 [310  73]]


In [ ]:
def value(solution):
    return sum(VALUES[i] for i in range(NUM_ITEMS) if solution[i] >= 0)


def is_feasible(solution):
    total_weights = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS), dtype=int)

    for i in range(NUM_ITEMS):
        k = solution[i]
        if k >= 0:
            total_weights[k] += WEIGHTS[i]

    for k in range(NUM_KNAPSACKS):
        for d in range(NUM_DIMENSIONS):
            if total_weights[k][d] > CONSTRAINTS[k][d]:
                return False

    return True


def simulated_annealing(max_iterations=10000, initial_temp=1000, cooling_rate=0.995):
    """Find a good solution to the generalized knapsack problem using simulated annealing.

    Args:
        max_iterations (int): Maximum number of iterations to perform.
        initial_temp (float): Initial temperature for the annealing process.
        cooling_rate (float): Rate at which the temperature decreases.
    """

    # Start with a random solution (which might not be feasible)
    current_solution = np.random.randint(-1, NUM_KNAPSACKS, size=NUM_ITEMS)

    while not is_feasible(current_solution):
        # Remove random items until feasible
        assigned_items = np.where(current_solution >= 0)[0]
        if len(assigned_items) == 0:
            break
        item_to_remove = np.random.choice(assigned_items)
        current_solution[item_to_remove] = -1

    current_value = value(current_solution)
    best_solution = current_solution.copy()
    best_value = current_value

    temp = initial_temp

    for _ in range(max_iterations):
        new_solution = current_solution.copy()
        item_to_change = np.random.randint(0, NUM_ITEMS)
        new_solution[item_to_change] = np.random.randint(-1, NUM_KNAPSACKS)

        if is_feasible(new_solution):
            new_value = value(new_solution)
            delta = new_value - current_value

            # Accepts if equal or better or with a probability based on temperature
            if delta >= 0 or np.random.rand() < np.exp(delta / max(temp, 1e-10)):
                current_solution = new_solution
                current_value = new_value

                if current_value > best_value:
                    best_solution = current_solution.copy()
                    best_value = current_value

        temp *= cooling_rate

    return best_solution, best_value

## TEST PROBLEMS

In [40]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [42]:
best_solution, best_value = simulated_annealing()
print("Problem 1:")
print(f"Best Value: {best_value}")
print(f"Best Solution: {best_solution}")

New best value found: 371
New best value found: 383
New best value found: 452
New best value found: 493
New best value found: 515
New best value found: 592
New best value found: 659
New best value found: 735
New best value found: 806
New best value found: 815
New best value found: 888
New best value found: 939
New best value found: 962
New best value found: 1025
New best value found: 1045
New best value found: 1053
Problem 1:
Best Value: 1053
Best Solution: [ 1  1  1  0  0  0  2  2  0  2  1  0  1  2  0  0  0 -1  1  0]


In [43]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [44]:
best_solution, best_value = simulated_annealing()
print("Problem 2:")
print(f"Best Value: {best_value}")
print(f"Best Solution: {best_solution}")

New best value found: 15449
New best value found: 15469
New best value found: 15823
New best value found: 16186
New best value found: 16474
New best value found: 16925
New best value found: 17692
New best value found: 18158
New best value found: 18844
New best value found: 19071
New best value found: 19147
New best value found: 19816
New best value found: 20452
New best value found: 20549
New best value found: 20861
New best value found: 21702
New best value found: 21856
New best value found: 22617
New best value found: 23271
New best value found: 23704
New best value found: 24091
New best value found: 24154
New best value found: 24546
New best value found: 25072
New best value found: 25831
New best value found: 26185
New best value found: 26622
New best value found: 26707
New best value found: 26801
New best value found: 27126
New best value found: 28066
New best value found: 28964
New best value found: 29158
New best value found: 29815
New best value found: 29997
New best value found

In [45]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10,
    1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS,
    size=(NUM_KNAPSACKS, NUM_DIMENSIONS),
)

In [46]:
best_solution, best_value = simulated_annealing()
print("Problem 3:")
print(f"Best Value: {best_value}")
print(f"Best Solution: {best_solution}")

New best value found: 424339
New best value found: 424572
New best value found: 425048
New best value found: 425394
New best value found: 426363
New best value found: 427135
New best value found: 427555
New best value found: 428335
New best value found: 428621
New best value found: 428724
New best value found: 429127
New best value found: 429259
New best value found: 429414
New best value found: 429742
New best value found: 429907
New best value found: 430227
New best value found: 430340
New best value found: 430610
New best value found: 430849
New best value found: 431525
New best value found: 432266
New best value found: 432291
New best value found: 432629
New best value found: 433403
New best value found: 434060
New best value found: 434708
New best value found: 435681
New best value found: 435917
New best value found: 435983
New best value found: 436262
New best value found: 436379
New best value found: 436857
New best value found: 437299
New best value found: 438159
New best value